# Frauenwahl
Am Tag 1 nach den Kantonsratswahlen mehrten sich die Hinweise, dass sich viele Frauen auf den Listen Plätze gut gemacht haben. Wir versuchten herauszufinden, ob dieser Eindruck stimmt.

In [ ]:
#my standard library import
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re
import time
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from cycler import cycler
plt.rcParams.update(
    {"figure.facecolor": "#ffffff",
              "axes.facecolor": "#ffffff",
              "axes.grid" : True,
              "axes.grid.axis" : "y",
              "grid.color"    : "ededed",
              "grid.linewidth": 0.8,
              "grid.alpha": 0.8,
              "axes.spines.left" : False,
              "axes.spines.right" : False,
              "axes.spines.top" : False,
              "axes.spines.bottom" : False,
              "axes.axisbelow": False,
              "ytick.major.size": 0,     
              "ytick.minor.size": 0,
              #"xtick.direction" : "none",
              "xtick.minor.size": 0,
              "xtick.major.size" : 0,
              "xtick.color"      : "#191919",
              "axes.edgecolor"    :"#191919",
              "axes.prop_cycle" : plt.cycler('color', ['#0c2c84', '#c7e9b4', 
                                                       '#225ea8','#1d91c0', '#41b6c4', 
                                                       '#7fcdbb', '#eaea8c']),
    'pdf.fonttype': 42,
    'ps.fonttype': 42
})
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

import glob
import json
from requests import get
from requests.auth import HTTPBasicAuth



In [ ]:
#####Liste Kandidierende
####JSON-Files des statistischen Amtes des Kanton Zürich herunterladen
url = "http://www.wahlen.zh.ch/wahlen2019/public/data/krw/kandidaten.json"
d = json.loads(requests.get(url).content)
d['kandidaten'][0]

In [ ]:
df = pd.DataFrame(d['kandidaten'])
#gewaehlt = df[df['gewaehlt']==1]
print(len(df))

In [ ]:
#listenplatz aus kandidatnnummer rauslesen und name erstellen
df['listen_platz']= df['kandidaten_nr'].str.extract('\d\d*_\d\d*(\d\d)').astype(int)
df['name'] = df['vorname'] + " " + df['nachname']

In [ ]:
#kandidierendenliste mit geschlecht einlesen
#Geschlecht fehlte im JSON
gender = pd.read_csv('KRW19.csv')
#gender.columns = ['nachname', 'vorname', 'gender','Plz', 'ListeKurzbez', 'Wahlkurzbezeichnung']
len(gender)
gender.head()

In [ ]:
#datensätze nach wahlkreis und kandidatennummer sortieren und zusammenfügen
gender = gender.sort_values(['Wahlkreis', 'Li-Ka'], ascending=False).reset_index()
df['lika'] = df['kandidaten_nr'].astype(str).str.replace('\d\d*_','').astype(int)
df = df.sort_values(['wahlkreis_bez', 'lika'], ascending=False).reset_index()
df = pd.concat([df, gender], axis=1)

In [ ]:
#Ranglisten Gewinne berechnen
df['vorwaertsdiff'] = df['listen_platz']-df['rang']
df['vorwaerts'] = df['listen_platz']>df['rang']

In [ ]:
df.head()

In [ ]:
df = df[['name','G','listen_code','rang','listen_platz', 
         'Wahlkreis', 'vorwaerts','vorwaertsdiff', 'gewaehlt', 
         'bisher', 'jahrgang','stimmentotal']]

In [ ]:
#liste aller frauen exportieren, sortiert nach Differenz
frauen = df[df['G']=='W']
frauen = frauen.sort_values('vorwaertsdiff', ascending=False)
#frauen.to_csv('frauenliste.csv', index=False)
#liste aller Männer exportieren – zum Vergleichen
men = df[df['G']=='M']
men = men.sort_values('vorwaertsdiff', ascending=False)
#men.to_csv('maennerliste.csv', index=False)

In [ ]:
#Wie hoch ist der Antiel der Frauen, die Plätze gut gemacht haben?
df[df['G']=='W'].vorwaerts.value_counts('normalize')

In [ ]:
#Durchschnittliche Veränderung aller Kandidierenden
print('Frauen: '+ str(df[df['G']=='W'].vorwaertsdiff.sum()/len(df[df['G']=='W'])))
#im schnitt alle frauen
print('Männer: ' + str(df[df['G']=='M'].vorwaertsdiff.sum()/len(df[df['G']=='M'])))

In [ ]:
#Wie hoch ist der Antiel der Männer, die Plätze gut gemacht haben?
df[df['G']=='M'].vorwaerts.value_counts('normalize')

In [ ]:
# Hatten Frauen einfach auch mehr Chancen, besser platziert zu sein. Waren sie insgesamt auf den schlechteren, relativen Listenplätzen?
#Funktion zur Berechnung der Listenlänge
def get_numberonlist(row):
    return len(df[(df['listen_code']==row['listen_code']) & (df['Wahlkreis']==row['Wahlkreis'])])

In [ ]:
df['listenlaenge'] = df.apply(get_numberonlist, axis=1)

In [ ]:
#relative Listenposition berechnen
df['relativ'] = df['listen_platz']/df['listenlaenge']

In [ ]:
df.to_csv('all.csv', index=False)

In [ ]:
#Es stellt sich ein Wert zwischen Wert zwischen etwas über 0 und 1 ein. 
#Männer und Frauen haben ählniche relative Listenplätze
df.groupby('G').relativ.sum()/df.groupby('G').relativ.count()

In [ ]:
#noch Kontrolle mit Median
df[df['G']=='W'].relativ.median()

In [ ]:
df[df['G']=='M'].relativ.median()

In [ ]:
#Grafische Analyse der Verteilung
df[df['G']=='W'].relativ.hist()

In [ ]:
df[df['G']=='M'].relativ.hist()

In [ ]:
#Subset für neues, gewähltes Parlament
gewaehlt = df[(df['gewaehlt']==1)]
len(gewaehlt)

In [ ]:
#Verteilung der Frauenquote nach Fraktion
gewaehlt.groupby('listen_code').G.value_counts('normalize').unstack()

In [ ]:
gewaehlt.groupby('listen_code').G.value_counts('').unstack()

In [ ]:
#wie schauts mit den neugewaehlten aus
neugewaehlt = df[(df['gewaehlt']==1) &(df['bisher']==0) & (df['G']=='W')]
neugewaehlt.groupby('listen_code').G.value_counts('normalize').unstack()

In [ ]:
neugewaehlt.groupby('listen_code').G.value_counts('').unstack()